<h1>Task 3</h1>

<h2>Imports</h2>

In [673]:
import math
import numpy as np
import pandas as pd
from random import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

## Read csv file

In [674]:
data = pd.read_csv('IrisData.txt')

## Task functions

In [675]:
# Sigmoid activation function
def sigmoid_derivative(x):
    f = 1.0 / (1.0 + np.exp(-1.0*x))
    return (f * (1.0 - f))

In [676]:
def sigmoid(x):
    return (1.0 / (1.0 + np.exp(-1.0*x)))

In [677]:
# Hyper Tangent activation function
def tanh_derivative(x):
    f = (2.0/(1.0 + np.exp(-2.0*x))-1.0)
    return (1.0 - (f**2))

In [678]:
def tanh(x):
    return (2.0/(1.0 + np.exp(-2.0*x))-1.0)

## Creating the neural network layers

- Each layer consists of an input matrix, a weight matrix and an error list.

    - The Input layer, is the x_train or test matrix.

    - The output layer, is a (3, 1) matirx.
    
    - For each of the hidden layers the dimensions are (numOf neurons in this layer, numOf neurons in the previous layer)

In [679]:
# network contains layers in the form of dictionaries, each layer consists of:
# numpy array of layer weights.
# numpy array of layer inputs.
# net value.
# gradient value.
# All values are randomly intialized.
def nn_setup(numOf_hidden_Layers, numOf_neurons):
    # adding the number of neurons of the input layer
    numOf_neurons.insert(0, data.shape[1])
    # a list of dictionaries of numpy arrays, holding the layers weights, inputs, net and gradient value.
    network = list()
    input_layer   = {'weights':  np.random.rand(numOf_neurons[0]),
                     'inputs': np.random.rand(numOf_neurons[0]), 
                     'net': np.random.rand(numOf_neurons[0], 1), 
                     'gradient': np.random.rand(numOf_neurons[0], 1)}
    input_layer['weights'] = input_layer['weights'].reshape(input_layer['weights'].shape[0], 1)
    input_layer['inputs'] = input_layer['inputs'].reshape(input_layer['inputs'].shape[0], 1)
    
    hidden_layers = [{'weights': np.random.rand(numOf_neurons[i], numOf_neurons[i - 1]),
                      'inputs': np.random.rand(numOf_neurons[i], numOf_neurons[i - 1]), 
                      'net': np.random.rand(numOf_neurons[i], 1), 
                      'gradient': np.random.rand(numOf_neurons[i], 1)} for i in range(1, len(numOf_neurons))]
    
    output_layer  = {'weights':  np.random.rand(4, numOf_neurons[-1]),
                     'inputs': np.random.rand(4, numOf_neurons[-1]), 
                     'net': np.random.rand(4, 1), 
                     'gradient': np.random.rand(4, 1)}
    
    network.append(input_layer)
    network += hidden_layers
    network.append(output_layer)
    #print(network)
    
    return network

<h2>Backpropagation</h2>

### First feed forward function

- use vector/matrix multiplication to calculate net value on each layer.
    - net  = sum(dot(layer_x, W.T))

In [680]:
# network contains layers in the form of dictionaries, each layer consists of:
# numpy array of layer weights.
# numpy array of layer inputs.
# net value.
# gradient value.
# All values are randomly intialized.
def feed_forward1(network, input_row, activation_fn, use_bias):
    # for each layer use vector/matrix multiplication to calculate the net value and update it in the network.
    network[0]['inputs'] = input_row.reshape(input_row.shape[0], 1)
    network[1]['inputs'] = input_row.reshape(input_row.shape[0], 1)
    last_output = []
    for i in range(1, len(network)):
        #if not use_bias:
        #    network[i]['weights'][:][0] = np.zeros(network[i]['weights'].shape[1])
        # calculate the product of the current layer's weights and inputs.
        neurons_val = np.dot(network[i]['weights'], network[i]['inputs'])
        # calculate the net value.
        network[i]['net'] = neurons_val
        
        # apply the activation function on the cur neurons values to get their output.
        if activation_fn is 'Sigmoid':
            neurons_val = [sigmoid(val) for val in neurons_val]
        else:
            neurons_val = [tanh(val) for val in neurons_val]
            
        # the next layer input is this layer's output
        if i < len(network) - 1:
            network[i + 1]['inputs'] = np.array(neurons_val).reshape(-1, 1)
        else:
            last_output = [1 if max(neurons_val[1:]) is neurons_val[i] else 0 for i in range(1, len(neurons_val))]
            last_output.insert(0, 1)
    return network, last_output

## First feed backward function

- use vector/matrix multiplication to calculate gradient value on each layer.
    - Output_layer_gradient = (intended_y - predicted_y) * d_activation_fn(net)
    - Hidden_layer_gradient_i = (gradient_(i-1) * W_i * d_activation_fn(net_i)

In [681]:
# network contains layers in the form of dictionaries, each layer consists of:
# numpy array of layer weights.
# numpy array of layer inputs.
# net value.
# gradient value.
# All values are randomly intialized.
def feed_backward(network, intended_y, predicted_y, activation_fn):
    # for each layer use vector/matrix multiplication to calculate the gradient value and update it in the network.
    # calculating gradient for output layer
    output_layer = network[-1]
    # clalculate the error cost between the predicted and the intended values.
    output_layer['gardient'] = intended_y - predicted_y
    # calculate the derivative activation of each net value.
    if activation_fn is 'Sigmoid':
        deriv = [sigmoid_derivative(val) for val in output_layer['net']]
    else:
        deriv = [tanh_derivative(val) for val in output_layer['net']]
    # reshaping for multiplication
    output_layer['gradient'].flatten()
    deriv = np.array(deriv).reshape(-1, 1)
    output_layer['gradient'] = output_layer['gradient'].reshape(-1, 1)
    # clalulating the output gradient = (intended_y - predicted_y) * acyivation_derivative(output_net_values)
    output_layer['gardient'] = output_layer['gardient'] * deriv
    #updating the gradient value in the network
    network[-1] =  output_layer
    
    # for each hidden layer, starting from the last one
    for i in range(len(network) - 2, 0, -1):
        # calculate the derivative activation of each net value.
        if activation_fn is 'Sigmoid':
            deriv = [sigmoid_derivative(val) for val in network[i]['net']]
        else:
            deriv = [tanh_derivative(val) for val in network[i]['net']]
        # reshaping for multiplication
        deriv = np.array(deriv).reshape(-1, 1)
        # calculating the sum of neuron weights between this layer and the previous one multiplied by the previous gradient.
        network[i]['gradient'] = np.dot(network[i + 1]['weights'].T, network[i + 1]['gradient']) * deriv
    return network
"""
def feed_backward(network, intended_y, predicted_y, activation_fn):
    # for each layer use vector/matrix multiplication to calculate the gradient value and update it in the network.
    # calculating gradient for output layer
    output_layer = network[len(network) - 1]
    output_layer['gradient'] = intended_y - predicted_y
    if activation_fn is 'Sigmoid':
        deriv = [sigmoid_derivative(val) for val in output_layer['net']]
    else:
        deriv = [tanh_derivative(val) for val in output_layer['net']]
    deriv = np.array(deriv).reshape(-1, 1)
    net = np.array(output_layer['net']).reshape(-1, 1)
    output_layer['gradient'].flatten()
    output_layer['gradient'] = output_layer['gradient'].reshape(-1, 1)
    output_layer['gradient'] = output_layer['gradient'] * deriv
    #np.dot(output_layer['gradient'], net)
    network[len(network) - 1] = output_layer
    print(len(network))
    # calculating gradient for hidden layers
    previous_gradient = output_layer['gradient']
    for i in range(len(network) - 2, -1, -1): # step=-1
        layer = network[i]
        next_layer = network[i + 1]
        #print('prev gradient', previous_gradient.shape)
        #print('weights', next_layer['weights'].shape)
        layer['gradient'] = np.dot(next_layer['weights'].T, next_layer['gradient'])
        if activation_fn is 'Sigmoid':
            deriv = [sigmoid_derivative(val) for val in layer['net']]
        else:
            deriv = [tanh_derivative(val) for val in layer['net']]
        deriv = np.array(deriv).reshape(-1, 1)
        net = np.array(layer['net']).reshape(-1, 1)
        #print('gradient', layer['gradient'].shape)
        #print('deriv', deriv.shape)
        layer['gradient'].flatten()
        #print('flatten gradient', layer['gradient'].shape)
        #print('\n')
        layer['gradient'] = layer['gradient'] * deriv
        print('\n')
        print('Network', i)
        print('cur grad', layer['gradient'])
        print('prev grad', next_layer['gradient'])
        print('weights', next_layer['weights'])
        print('deriv', deriv)
        #np.dot(layer['gradient'], net)
        network[i]['gradient'] = layer['gradient']  
        previous_gradient = layer['gradient']
    #print('backward net', network[1]['net'])
    return network
    """

"\ndef feed_backward(network, intended_y, predicted_y, activation_fn):\n    # for each layer use vector/matrix multiplication to calculate the gradient value and update it in the network.\n    # calculating gradient for output layer\n    output_layer = network[len(network) - 1]\n    output_layer['gradient'] = intended_y - predicted_y\n    if activation_fn is 'Sigmoid':\n        deriv = [sigmoid_derivative(val) for val in output_layer['net']]\n    else:\n        deriv = [tanh_derivative(val) for val in output_layer['net']]\n    deriv = np.array(deriv).reshape(-1, 1)\n    net = np.array(output_layer['net']).reshape(-1, 1)\n    output_layer['gradient'].flatten()\n    output_layer['gradient'] = output_layer['gradient'].reshape(-1, 1)\n    output_layer['gradient'] = output_layer['gradient'] * deriv\n    #np.dot(output_layer['gradient'], net)\n    network[len(network) - 1] = output_layer\n    print(len(network))\n    # calculating gradient for hidden layers\n    previous_gradient = output_la

### Second feed forward function

- use vector/matrix multiplication to Update the weight matrix in each layer.
    - W_i = W_i + (learning_rate * gradient_i * x_i)

In [682]:
# network contains layers in the form of dictionaries, each layer consists of:
# numpy array of layer weights.
# numpy array of layer inputs.
# net value.
# gradient value.
# All values are randomly intialized.
def feed_forward2(network, learning_rate):
    # for each layer use vector/matrix multiplication to calculate the new weights value and update it in the network.
    for i in range (1 , len(network)):
        
        if network[i]['inputs'].shape[0] < network[i]['weights'].shape[0]:
            # updating the weights with the given input
            for j in range(network[i]['inputs'].shape[0]):
                network[i]['weights'][j] += network[i]['inputs'][j] * learning_rate * network[i]['gradient'][j]
            # updating the rest of the weights with no.
            for j in range(network[i]['inputs'].shape[0], network[i]['weights'].shape[0]):
                network[i]['weights'][j] += learning_rate * network[i]['gradient'][j]
        else:
            # updating the weights with the given input
            for j in range(network[i]['weights'].shape[0]):
                network[i]['weights'][j] += network[i]['inputs'][j] * learning_rate * network[i]['gradient'][j]
        
    return network


"""
def feed_forward2(network, learning_rate):
     # for each layer use vector/matrix multiplication to calculate the new weights value and update it in the network.
    for i in range (1 , len(network)):
        layer = network[i]
        #print('gradient1', layer['gradient'])
        #print('weights1', layer['weights'])
        net = (layer['inputs'] * layer['gradient']) * learning_rate
        print('net', net.shape)
        print('gradient', layer['gradient'].shape)
        print('net', layer['net'].shape)
        net = net.reshape(-1, 1)
        
        for neuron_index in range(layer['weights'].shape[0]):
            layer['weights'][neuron_index] += net[neuron_index]
        #print('weights2', layer['weights'])
        network[i] = layer
        
    return network
"""

"\ndef feed_forward2(network, learning_rate):\n     # for each layer use vector/matrix multiplication to calculate the new weights value and update it in the network.\n    for i in range (1 , len(network)):\n        layer = network[i]\n        #print('gradient1', layer['gradient'])\n        #print('weights1', layer['weights'])\n        net = (layer['inputs'] * layer['gradient']) * learning_rate\n        print('net', net.shape)\n        print('gradient', layer['gradient'].shape)\n        print('net', layer['net'].shape)\n        net = net.reshape(-1, 1)\n        \n        for neuron_index in range(layer['weights'].shape[0]):\n            layer['weights'][neuron_index] += net[neuron_index]\n        #print('weights2', layer['weights'])\n        network[i] = layer\n        \n    return network\n"

### Backpropagation model

- For each epoch call:

    - Feed forward, calculating the net values for each layer.

    - Feed Backward, calculating the gradient values for each layer.

    - Feed forward, updating the weights for each layer.

In [683]:
def backpropagation(x_train, y_train, network, learning_rate, epochs, use_bias, activation_fn):
    # for each epoch:
    for i in range(epochs):
        for ind in range(x_train.shape[0]):
            # call feed_forward1 with the given network, row of data, activation function and use_bias.
            network, y_prediction = feed_forward1(network, x_train[ind], activation_fn, use_bias)
            # call feed_backward with the returned network, cur row of y_train, cur y_prediction for this row and the activation_fn
            network = feed_backward(network, y_train[ind], np.array(y_prediction), activation_fn)
            # call feed_forward2 with the returned network and other necessary values
            network = feed_forward2(network, learning_rate)
    return network

## Testing

In [684]:
def test(x_test, y_test, network, activation_fn, use_bias):
    y_prediction = []
    for row in x_test:
        network, y = feed_forward1(network, row, activation_fn, use_bias)
        y_prediction.append(y)
    print(y_prediction)
    # calculating the accuracy.
    comparison = (y_prediction == y_test)
    co = 0
    for l in comparison:
        ans = True
        for val in l:
            ans &= val
        if ans == True:
            co += 1
    accuracy = (co/y_test.shape[0]) * 100
    return y_prediction, accuracy

## Extracting the data

In [685]:
# We will be using all the 4 feauters and 3 classes.
# The y column should be on hot encoded, meaning that if the label is c1 
    # then it should be represented as follow, 100 and so on.

def extract_data():
    data_x = data.iloc[:, :4]
    x0 = np.ones([150, 1]) # feature 0 for bias
    data_x = np.append(x0, data_x, axis=1)
    # One hot encoding the ouput column.
    values = np.array(data['Class'])
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    data_y = onehot_encoded
    data_y = np.append(x0, data_y, axis=1)
    
    x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.4, shuffle=True, stratify = data_y)
    return x_train, y_train, x_test, y_test


## Main

In [686]:
def main(numOf_hidden_layers, numOf_nuerons, activation_fn, learning_rate, epochs, use_bias):
    # Get the train and test data.
    x_train, y_train, x_test, y_test = extract_data()
    
    # Setup the Neural Network layer.
    network = nn_setup(numOf_hidden_layers, numOf_nuerons)
    
    #print(network)
    
    # Call the backpropagation model and return the learned weights.
    network = backpropagation(x_train, y_train, network, learning_rate, epochs, use_bias, activation_fn)

    print(network)
    
    # Test the moddel and return its accuracy, then print it.
    y_prediction_test, accuracy_test = test(x_test, y_test, network, activation_fn, use_bias)
    print('Testing accuracy:\n', accuracy_test)
    
    # print the confusion matrix.
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_prediction_test))
    return

<h1>UI</h1>

- text box to take the number of desired hidden layers.

- text box to take the number of desired neurons in each hidden layer.

- combo box to choose the activation function, sigmoid or tanh.

- text box to take the desired learning rate.

- text box to take the desired number of epochs.

- check box for the bias.

In [687]:
from tkinter import *

In [688]:
input_window = Tk()
input_window.title("Neural Networks Task 3")
input_window.geometry("500x500")
activation_fns = ["Sigmoid", "Tanh"]

<h3>Number of Hidden Layers</h3>

In [689]:
#Number of hidden Layers label
numOf_hidden_layers_value = StringVar()
numOf_hidden_layers_label = Label(input_window, textvariable = numOf_hidden_layers_value) 
numOf_hidden_layers_value.set("Number of hidden Layers")
numOf_hidden_layers_label.place(x=45, y=100)
#Number of hidden Layers text
numOf_hidden_layers_text = Entry(input_window)
numOf_hidden_layers_text.place(x=190, y=100)
numOf_hidden_layers_text.focus_set()

<h3>Number of neurons in each hidden layer</h3>

In [690]:
#Number of neurons hidden Layers label
numOf_neurons_value = StringVar()
numOf_neurons_label = Label(input_window, textvariable = numOf_neurons_value) 
numOf_neurons_value.set("Number of neurons in them")
numOf_neurons_label.place(x=30, y=140)
#Number of neurons hidden Layers text
numOf_neurons_text = Entry(input_window)
numOf_neurons_text.place(x=190, y=140)
numOf_neurons_text.focus_set()

<h3>Activation function Dropdown List</h3>

In [691]:
#Activation fn label
activation_fn_value = StringVar()
activation_fn_label = Label(input_window, textvariable = activation_fn_value) 
activation_fn_value.set("Activaion function")
activation_fn_label.place(x=80, y=170)
#Activation fn list
activation_fn_var = StringVar(input_window)
activation_fn = OptionMenu(input_window, activation_fn_var, *activation_fns)
activation_fn.config(width=12, font=('Helvetica', 10))
#activation_fn_var.set('Sigmoid') # set the default option
activation_fn.place(x=190, y=170)

<h3>Learning Rate</h3>

In [692]:
#learning rate label
learning_rate_value = StringVar()
learning_rate_label = Label(input_window, textvariable = learning_rate_value) 
learning_rate_value.set("Learning Rate")
learning_rate_label.place(x=105, y=210)
#learning rate text
learning_rate_text = Entry(input_window)
learning_rate_text.place(x=193, y=210)
learning_rate_text.focus_set()

<h3>Epochs</h3>

In [693]:
#Epochs label
epochs_label_value = StringVar()
epochs_label = Label(input_window, textvariable = epochs_label_value) 
epochs_label_value.set("Epochs")
epochs_label.place(x=140, y=240)
#Epochs text
epochs_text = Entry(input_window)
epochs_text.place(x=193, y=240)
epochs_text.focus_set()

<h3>Bias</h3>

In [694]:
#Bias check box
bias_checkbox = IntVar()
Checkbutton(input_window, text="Bias", variable=bias_checkbox).place(x=190,y=290)

<h3>Training The Model Button</h3>

In [695]:
def submit_button_backpropagation():
    layers_neurons = numOf_neurons_text.get().split()
    layers_neurons = [int(val) for val in layers_neurons]
    main(int(numOf_hidden_layers_text.get()), layers_neurons, activation_fn_var.get(), float(learning_rate_text.get()),
         int(epochs_text.get()), int(bias_checkbox.get()))


In [696]:
#Button
train_model_button = Button(input_window, text='Train backpropagation', width=17, command=submit_button_backpropagation)
train_model_button.place(x=190, y=320)

In [697]:
input_window.mainloop() #open window

[{'weights': array([[0.46593822],
       [0.74848826],
       [0.43659647],
       [0.07537571],
       [0.92481155]]), 'inputs': array([[1. ],
       [6.6],
       [3. ],
       [4.4],
       [1.4]]), 'net': array([[0.46507271],
       [0.81943151],
       [0.14289054],
       [0.87222164],
       [0.37749624]]), 'gradient': array([[0.95366997],
       [0.5930361 ],
       [0.7428768 ],
       [0.20793113],
       [0.27432674]])}, {'weights': array([[0.37115869, 0.4512833 , 0.51757273, 0.24439016, 0.67199213],
       [0.65175893, 0.5689809 , 0.60340862, 0.43864958, 0.46201803],
       [0.29832985, 0.90544861, 0.26269428, 0.14738842, 0.05887481],
       [0.81234719, 0.21868068, 0.70222992, 0.01089333, 0.77308089],
       [0.67530206, 0.05500928, 0.7842008 , 0.46480409, 0.4705675 ],
       [0.41377955, 0.56805927, 0.13066895, 0.93152405, 0.46933826],
       [0.72707014, 0.4652443 , 0.27089285, 0.05021525, 0.03891764],
       [0.45989133, 0.06696804, 0.85074451, 0.95274228, 0.56727208],


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-695-75233577d4ed>", line 5, in submit_button_backpropagation
    int(epochs_text.get()), int(bias_checkbox.get()))
  File "<ipython-input-686-42bf10e28043>", line 20, in main
    print('Confusion Matrix:\n', confusion_matrix(y_test, y_prediction_test))
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py", line 253, in confusion_matrix
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py", line 81, in _check_targets
    "and {1} targets".format(type_true, type_pred))
ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass-multioutput targets
